In [60]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Install Postgres Driver
!wget https://jdbc.postgresql.org/download/postgresql-42.5.0.jar

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:14 http://archive.ubuntu.com/ubuntu focal-updates/universe 

In [61]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.5.0.jar

--2023-02-11 01:23:44--  https://jdbc.postgresql.org/download/postgresql-42.5.0.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1046274 (1022K) [application/java-archive]
Saving to: ‘postgresql-42.5.0.jar.6’

postgresql-42.5.0.j 100%[===================>]   1022K  6.56MB/s    in 0.2s    

2023-02-11 01:23:45 (6.56 MB/s) - ‘postgresql-42.5.0.jar.6’ saved [1046274/1046274]



In [62]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.5.0.jar").getOrCreate()

Load Amazon Data into Spark DataFrame

In [63]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

# Show DataFrame
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...|2015-08-31 00:00:00|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...| 

Create DataFrames

In [64]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
amazonReview_df = df.select(["review_id","customer_id","product_id","product_parent","product_title","review_date","star_rating", "helpful_votes","total_votes","vine","verified_purchase"])
amazonReview_df.show(5)

+--------------+-----------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+
|     review_id|customer_id|product_id|product_parent|       product_title|        review_date|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+
|R3W4P9UBGNGH1U|   16414143|B00YL0EKWE|     852431543|LG G4 Case Hard T...|2015-08-31 00:00:00|          2|            1|          3|   N|                Y|
|R15V54KBMTQWAY|   50800750|B00XK95RPQ|     516894650|Selfie Stick Fibl...|2015-08-31 00:00:00|          4|            0|          0|   N|                N|
| RY8I449HNXSVF|   15184378|B00SXRXUKO|     984297154|Tribe AB40 Water ...|2015-08-31 00:00:00|          5|            0|          0|   N|                Y|
|R18TLJYCKJFLSR|   10203548|B009V5X1CE|     279912704|RAVP

In [65]:
# Create the vine_table. DataFrame
vine_df = amazonReview_df.select(["review_id","star_rating", "helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|                Y|
|R15V54KBMTQWAY|          4|            0|          0|   N|                N|
| RY8I449HNXSVF|          5|            0|          0|   N|                Y|
|R18TLJYCKJFLSR|          5|            0|          0|   N|                Y|
|R1NK26SWS53B8Q|          5|            0|          0|   N|                Y|
|R11LOHEDYJALTN|          5|            0|          0|   N|                Y|
|R3ALQVQB2P9LA7|          4|            0|          0|   N|                Y|
|R3MWLXLNO21PDQ|          5|            0|          0|   N|                Y|
|R2L15IS24CX0LI|          5|            0|          0|   N|                Y|
|R1DJ8976WPWVZU|          3|            0|          0|   N|     

In [66]:
# create a new DataFrame where the total_votes count is equal to or greater than 20
vine_vote_df = vine_df.filter("total_votes >= 20")
vine_vote_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|
|R2EE2TR4MRDV0U|          5|           53|         59|   N|                Y|
| RD4A80I5JDHED|          5|           69|         77|   N|                N|
|R1GU6IYZQWQE8X|          2|           24|         25|   N|     

In [67]:
# create a new DataFrame or table to retrieve all the rows 
# where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

helpfulVotes_df = vine_vote_df.withColumn('helpful_votes_%',vine_vote_df['helpful_votes']/vine_vote_df['total_votes'])
votePercent_df = helpfulVotes_df.filter(helpfulVotes_df["helpful_votes_%"]  >=.5).show()

#helpful_votes_df = vine_vote_df.filter((vine_vote_df["helpful_votes"] / vine_vote_df["total_votes"]) >= .5)
#helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   helpful_votes_%|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|0.9152542372881356|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|0.7142857142857143|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|0.9523809523809523|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|0.6363636363636364|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|0.9574468085106383|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|0.9423076923076923|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|0.6666666666666666|
|R2EE2TR4MRDV0U|          5|  

In [68]:
# create a new DataFrame or table that retrieves all the rows 
# where a review was written as part of the Vine program (paid), vine == 'Y'

paid_vine_df = vine_vote_df.filter(vine_vote_df["vine"]=="Y")
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1MAOLI5FJHAFM|          4|          249|        261|   Y|                N|
| R9PYAUDIBJVEC|          4|           12|         22|   Y|                N|
|R2DOTPESCGMW6A|          4|            5|         24|   Y|                N|
| R6V9SHMMG5M8F|          5|          101|        110|   Y|                N|
|R37PVLT6ELL5J4|          4|          181|        209|   Y|                N|
|R1IG7IRDCIKJBN|          5|            9|         48|   Y|                N|
| R2FSFGWZF24V9|          4|           50|         51|   Y|                N|
|R3SRW1E8J56IGV|          5|          262|        281|   Y|                N|
| R86Z11D4CWOFM|          4|           32|         36|   Y|                N|
| RNP01HW9YISJO|          4|           20|         23|   Y|     

In [69]:
# create a new DataFrame or table that retrieves all the rows 
# where a review was written as part of the Vine program (paid), vine == 'N'

unpaid_vine_df = vine_vote_df.filter(vine_vote_df["vine"]=="N")
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|
|R2EE2TR4MRDV0U|          5|           53|         59|   N|                Y|
| RD4A80I5JDHED|          5|           69|         77|   N|                N|
|R1GU6IYZQWQE8X|          2|           24|         25|   N|     

In [70]:
# total number of reviews (paid vs unpaid)

paid_review_df = paid_vine_df.count()
paid_review_df 

618

In [71]:
unpaid_vine_df = unpaid_vine_df.count()
unpaid_vine_df 

71969

In [73]:
# number of 5-star reviews (paid vs unpaid)
paid_fiveStar_df = paid_vine_df.filter(paid_vine_df["star_rating"]==5).count()
paid_fiveStar_df

223

In [89]:
#unpaid_fiveStar_df = unpaid_vine_df.filter(unpaid_vine_df["star_rating"]== 5).count()
#unpaid_fiveStar_df

unpaid_fiveStar_df = vine_vote_df.filter((vine_vote_df["vine"]=="N") & (vine_vote_df["star_rating"]== 5))
unpaid_fiveStar_df.count()

31390

In [83]:
# the percentage of 5-star reviews
total_fiveStar_review = vine_vote_df.filter(paid_vine_df["star_rating"]==5).count()
total_fiveStar_review

31613

In [85]:
#1 the percentage of 5-star reviews for the two types of review (paid vs unpaid).
paid_fiveStar_percent_df = (paid_fiveStar_df/total_fiveStar_review)*100
paid_fiveStar_percent_df

0.7054060038591718

In [97]:
#2 the percentage of 5-star reviews for the two types of review (paid vs unpaid).
unpaid_fiveStar_percent_df = (unpaid_fiveStar_df/total_fiveStar_review)*100
unpaid_fiveStar_percent_df